## Mastering Machine Learning 2025

Taller 2: tokenizers y embeddings

Antes de iniciar abra este cuaderno en Google Colab y habilite la ejecución con GPU:
- En el menú Entorno de ejecución seleccione Cambiar tipo entorno de ejecución.
- Asegúrese de tener seleccionado Python 3.
- Como Acelerador de hardware seleccione GPU T4.

Instale las dependencias para asegurar la correcta ejecución del cuaderno.

In [ ]:
!pip install --upgrade transformers==4.41.2 sentence-transformers==3.0.1 gensim==4.3.2 scikit-learn==1.5.0 accelerate==0.31.0 peft==0.11.1 scipy==1.10.1 numpy==1.26.4

## Generar texto con prompts y tokens

Ahora usaremos la librería transformers de Hugging Face. Puede crear una cuenta en https://huggingface.co/

Importamos las clases para el tokenizador y el modelo de lenguaje Phi-3-Mini-4K-Instruct, disponible en Hugging Face.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Cargamos el modelo
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=False,
)

# Cargamos el tokenizador
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

Ejecutemos un prompt solicitando un correo

In [ ]:
# Prompt a ejecutar
prompt = "Write an email to authorize the entrance of King William to the University.<|assistant|>"

# Tokenizar el prompt de entrada - retorna tensores de pytorch - los tipos de datos son compatibles con CUDA
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

# Generar el texto
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=40
)

# Imprimir la salida (decodificada)
print(tokenizer.decode(generation_output[0]))

Revisemos los tokens generados como entrada para el modelo, a partir del prompt

In [ ]:
input_ids

Revisemos la traducción de tokens a texto (decode)

In [ ]:
for id in input_ids[0]:
   print(f'{id}: {tokenizer.decode(id)}')

Revisemos la salida en formato token

In [ ]:
generation_output

Veamos el tipo de dato de esta salida

In [ ]:
generation_output[0][0]

Veamos ahora la traducción de tokens a texto (decode)

In [ ]:
for id in generation_output[0]:
   print(f'{id}: {tokenizer.decode(id)}')

## Comparar tokenizadores

Usaremos la siguiente función para ilustrar el resultado de la tokenización

In [ ]:
colors_list = [
    '102;194;165', '252;141;98', '141;160;203',
    '231;138;195', '166;216;84', '255;217;47'
]

def show_tokens(sentence, tokenizer_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids
    for idx, t in enumerate(token_ids):
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
            tokenizer.decode(t) +
            '\x1b[0m',
            end=' '
        )

Usaremos este texto de prueba

In [ ]:
text = """

English and CAPITALIZATION

🎵鸟
show_tokens False None elif == >= else: two tabs:" " Three tabs: "   "

12.0*50=600

"""

Inicialmente usaremos BERT en su versión uncased (2018)

In [ ]:
show_tokens(text, "google-bert/bert-base-uncased")

Ahora usaremos BERT en su versión cased (2018)

In [ ]:
show_tokens(text, "google-bert/bert-base-cased")

Ahora usamos GPT-2 (2019)

In [ ]:
show_tokens(text, "openai-community/gpt2")

Ahora usamos Flan-T5 (2022)

In [ ]:
show_tokens(text, "google/flan-t5-xxl")

Ahora usamos StarCoder2 (2024)

In [ ]:
show_tokens(text, "bigcode/starcoder2-15b")

Ahora usamos Phi-3 que usamos al inicio y es similar al de Llama 2

In [ ]:
show_tokens(text, "microsoft/Phi-3-mini-4k-instruct")